In [4]:
import numpy as np

In [11]:
n = int(1e6)

In [43]:
seeded = 
counter = 0

In [44]:
def roll2D6():
    global counter
    counter += 1
    return seeded[counter]

In [45]:
def inj(mb=False, po=False):
    
    result = roll2D6() + mb
    if result > 
    
    
    return result
    

In [53]:
import pandas as pd

In [182]:
# function for no skills:
def noskill(av, n=1000000):

    dr = np.random.randint(1, 7, n) + np.random.randint(1, 7, n)
    dr2 = np.random.randint(1, 7, n) + np.random.randint(1, 7, n)
    r = np.where(dr <= av, 0, np.where(dr2 <= 7, 1, np.where(dr2 <= 9, 2, 3)))
    
    # count 0/1/2/3
    return np.array(pd.value_counts(r, normalize=True))

# function for mb/claw
def mb(av, n=1000000, norm=True):
    
    dr = np.random.randint(1, 7, n) + np.random.randint(1, 7, n)
    dr2 = np.random.randint(1, 7, n) + np.random.randint(1, 7, n)
    
    # use mb
    mb_av = (dr - av == 0)
    dr += mb_av
    dr2 += ~mb_av
    
    r = np.where(dr <= av, 0, np.where(dr2 <= 7, 1, np.where(dr2 <= 9, 2, 3)))
    
    # count 0/1/2/3
    return np.array(pd.value_counts(r, normalize=norm))

# function for mb/claw
def clawmb(av, n=1000000):        
    return mb(min(av, 7))

# function for mb/po (always unless CAS/KO)
# function for mb/claw
def mbpo(av, n=1000000, claw=False):
    
    if claw:
        av = min(av, 7)
    
    avroll = np.random.randint(1, 7, n) + np.random.randint(1, 7, n)
    injroll = np.random.randint(1, 7, n) + np.random.randint(1, 7, n)
    poroll = np.random.randint(1, 7, n) + np.random.randint(1, 7, n)
    
    # use mb
    r = np.zeros(n)
    
    result = np.sort(avroll - av)
    
    # how many below 0? These get po, same as mb
    fail = np.sum(result < 0)
    
    #print("failed", fail)
    fail_r = mb(av=av, n=fail, norm=False)
    
    needmb = np.sum(result == 0)
    #print("Need mb", needmb)
    # simple po on injury
    # if inj_roll > 7, keep else apply po.
    result_need = np.sort(np.where(injroll[:needmb] > 7, injroll[:needmb], poroll[:needmb]))
    #print(result_need.mean(), injroll.mean(), poroll.mean())
    ca = pd.value_counts(np.where(result_need <= 7, 1, np.where(result_need <= 9, 2, 3)))
        
    thru = np.sum(result > 0)
    #print("straight thru", thru)
    # simple po on injury with + 1
    
    # if inj_roll > 7, keep else apply po.
    result_su = np.sort(np.where(injroll[:thru] >= 7, injroll[:thru], poroll[:thru]) + 1)
    cb = pd.value_counts(np.where(result_su <= 7, 1, np.where(result_su <= 9, 2, 3)))
    
    # count 0/1/2/3
    r = np.vstack([fail_r, np.insert(np.array(ca), 0, 0), np.insert(np.array(cb), 0, 0)])
    #print(fail, needmb, thru)
    
    return r.sum(axis=0)/r.sum()

In [167]:
injroll = np.random.randint(1, 7, n) + np.random.randint(1, 7, n)
poroll = np.random.randint(1, 7, n) + np.random.randint(1, 7, n)

In [168]:
result_need = np.where(injroll > 7, injroll, poroll)


In [176]:
pd.value_counts(np.where(result_need <= 7, 1, np.where(result_need <= 9, 2, 3)))

2    395840
1    340473
3    263687
dtype: int64

In [164]:
qq = np.array(range(10))




In [165]:
np.where(qq < 5, qq + 100, qq)

array([100, 101, 102, 103, 104,   5,   6,   7,   8,   9])

In [183]:
mbpo(7)

2    66118
1    56736
3    43863
dtype: int64
2    66118
1    56736
3    43863
dtype: int64
2    180036
3    163834
1     72661
dtype: int64


array([ 0.17366 ,  0.359047,  0.291021,  0.176272])

In [139]:
rows = xx.sum(axis=1)
rows

array([416525, 166354, 417121])

In [143]:
xx

array([[173709, 112642,  70286,  59888],
       [     0,  66107,  56496,  43751],
       [     0, 180509, 163891,  72721]])

In [144]:
xx.T/rows

array([[ 0.41704339,  0.        ,  0.        ],
       [ 0.27043275,  0.3973875 ,  0.43274973],
       [ 0.16874377,  0.33961311,  0.39290997],
       [ 0.14378009,  0.26299939,  0.1743403 ]])

In [83]:
AV = [7, 8, 9, 10]
result = ["no break", "stun", "KO", "CAS"]

In [88]:
big_n = 10000000

In [89]:
no_skill = pd.DataFrame([noskill(x, big_n) for x in AV], columns=result, index=AV)
no_skill

,no break,stun,KO,CAS
7,0.583201,0.243331,0.104038,0.069431
8,0.722142,0.162097,0.069481,0.046280
9,0.833310,0.097273,0.041603,0.027815
10,0.916587,0.048648,0.020880,0.013885


In [90]:
mb_only = pd.DataFrame([mb(x, big_n) for x in AV], columns=result, index=AV)
mb_only

,no break,stun,KO,CAS
7,0.416544,0.270985,0.168905,0.143567
8,0.583303,0.196923,0.119639,0.100135
9,0.722059,0.134347,0.078692,0.064902
10,0.833282,0.083428,0.046287,0.037003


In [91]:
mb_claw = pd.DataFrame([clawmb(x, big_n) for x in AV], columns=result, index=AV)
mb_claw

,no break,stun,KO,CAS
7,0.416389,0.271316,0.168945,0.143350
8,0.416463,0.270722,0.169243,0.143572
9,0.416043,0.271273,0.169343,0.143341
10,0.416363,0.270930,0.169007,0.143700


In [134]:
mb_po = pd.DataFrame([mbpo(x, big_n) for x in AV], columns=result, index=AV)
mb_po

,no break,stun,KO,CAS
7,0.173451,0.359260,0.291157,0.176131
8,0.340154,0.289885,0.226519,0.143442
9,0.521254,0.213357,0.160274,0.105115
10,0.694368,0.138390,0.099824,0.067418
